In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross
from backtesting.test import SMA
import matplotlib.pyplot as plt
import talib as ta

c:\Coding\Algotrading\backtesting.py\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
# Get OHLCV data from Yahoo Finance
he = yf.download("HE", interval="15m", multi_level_index=False)
he.head()

[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Datetime,,,,,
2024-12-20 14:30:00+00:00,9.868242,9.890,9.590,9.5900,488649
2024-12-20 14:45:00+00:00,9.960000,9.980,9.850,9.8700,246591
2024-12-20 15:00:00+00:00,10.030000,10.050,9.965,9.9800,186299
2024-12-20 15:15:00+00:00,10.055000,10.085,10.000,10.0405,189224
2024-12-20 15:30:00+00:00,10.035000,10.120,10.010,10.0600,390947


In [7]:
def std_3(arr, n):
    return pd.Series(arr).rolling(n).std() * 3

class MeanReversion(Strategy):
    roll = 50

    def init(self):
        self.he = self.data.Close

        self.he_mean = self.I(SMA, self.he, self.roll)
        self.he_std = self.I(std_3, self.he, self.roll)
        self.he_upper = self.he_mean + self.he_std
        self.he_lower = self.he_mean - self.he_std

        self.he_close = self.I(SMA, self.he, 1)
    
    def next(self):

        if self.he_close < self.he_lower:
            self.buy(
                tp = self.he_mean,
            )

        if self.he_close > self.he_upper:
            self.sell(
                tp = self.he_mean,
            )

In [8]:
bt = Backtest(he, MeanReversion, cash=10000, commission=0.002)
stats = bt.run()
bt.plot()
stats

c:\Coding\Algotrading\backtesting.py\.venv\Lib\site-packages\bokeh\util\serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


Start                     2024-12-20 14:30...
End                       2025-02-14 20:45...
Duration                     56 days 06:15:00
Exposure Time [%]                    70.81138
Equity Final [$]                  10784.79934
Equity Peak [$]                     11139.359
Commissions [$]                      82.18405
Return [%]                            7.84799
Buy & Hold Return [%]                 2.31854
Return (Ann.) [%]                    67.29262
Volatility (Ann.) [%]                88.14731
CAGR [%]                             40.27196
Sharpe Ratio                          0.76341
Sortino Ratio                         1.96138
Calmar Ratio                          4.56091
Max. Drawdown [%]                   -14.75422
Avg. Drawdown [%]                    -5.16116
Max. Drawdown Duration       19 days 01:30:00
Avg. Drawdown Duration        5 days 09:32:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                    